In [1]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from RP_Downstream_Trainer import DownstreamNet, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from RP_Downstream_Trainer import smallest_class_len, restrict_training_size_per_class, train_end_to_end
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import LeaveOneGroupOut, GroupKFold
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset


In [2]:
def train_different_classes(model_path, tset, vset, epochs, sample_list):
    outList=[]
    balanced_acc_out=[]
    for i in sample_list:
        # print(i)
        acc, balanced_acc = train_end_to_end(model_path, tset, vset, i, epochs, 3)
        outList.append(acc)
        balanced_acc_out.append(balanced_acc)
    return outList, balanced_acc_out

# Pick the model name

In [3]:
model_path = "RP_stagernet_30_120.pth"

In [4]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    d = Downstream_Dataset(path=data_file)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
    
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(groups.shape)


# logo = LeaveOneGroupOut()
kfold = GroupKFold(n_splits=5)
# logo.get_n_splits(x_vals, y_vals, groups)
kfold.get_n_splits(x_vals, y_vals, groups)


result_dict = {}
# dtype=torch.int32
for train_index, test_index in kfold.split(x_vals, y_vals, groups):
    unique = np.unique(groups[test_index])
    # group_num = groups[test_index][0]
    print("Leaving out mouse number:", unique)
    training_set = TensorDataset(torch.tensor(x_vals[train_index], dtype=torch.float), torch.tensor(y_vals[train_index], dtype=torch.long))
    test_set = TensorDataset(torch.tensor(x_vals[test_index], dtype=torch.float), torch.tensor(y_vals[test_index], dtype=torch.long))
    print(len(training_set))
    smallest_class = smallest_class_len(training_set, 3)
    num_samples=[]
    temp=1
    while temp < smallest_class:
        num_samples.append(temp)
        temp*=10
    num_samples.append(None)
    
    

    
    accuracy, balanced_accuracy = train_different_classes(model_path, training_set, test_set, 100, num_samples)
    for num_pos, acc in zip(num_samples, accuracy):
        if num_pos not in result_dict:
            result_dict[num_pos] = []
        result_dict[num_pos].append(acc)
    print(result_dict)
    
for k in result_dict:
    result_dict[k] = np.mean(result_dict[k])
print(result_dict)


Loading Data
Processing MouseCKA1_030515_HCOFTS
removed 0 unknown entries
Processing MouseCKB9_022715_HCOFTS
removed 0 unknown entries
Processing MouseCKL1_062514_HCOFTS
removed 0 unknown entries
Processing MouseCKL5_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKL7_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKN1_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKN2_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN3_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKO1_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKO2_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKO3_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKR1_082514_HCOFTS
removed 0 unknown entries
Processing MouseCKR3_082514_HCOFTS
removed 0 unknown entries
Processing MouseCKU10_092514_HCOFTS
removed 0 unknown entries
Processing MouseCKU1_092414_HCOFTS
removed 0 unknown entries
Processing MouseCKV10_101414_HCOFTS
removed 0 unknown entries
Processin

In [4]:
gc.collect()

trial 1
{1: 0.5801833333333334, 10: 0.7514333333333333, 100: 0.8498000000000001, 1000: 0.9018, None: 0.9062333333333333}
trial 2
{1: 0.55125, 10: 0.72645, 100: 0.8608833333333331, 1000: 0.8951666666666667, None: 0.9088833333333334}
trial 3
{1: 0.5833833333333334, 10: 0.7462333333333333, 100: 0.8501999999999998, 1000: 0.8996999999999999, None: 0.9077333333333332}


60

In [1]:
trial1=[0.5801833333333334, 0.7514333333333, 0.8498000000000001, 0.9018, 0.9062333333333333]
trial2=[0.55125, 0.72645, 0.8608833333333331, 0.8951666666666667, 0.9088833333333334]
trial3=[0.5833833333333334, 0.7462333333333333, 0.8501999999999998, 0.8996999999999999, 0.9077333333333332]

In [3]:
trial1=[0.5801833333333334, 0.7514333333333, 0.8498000000000001, 0.9018, 0.9062333333333333]
trial2=[0.55125, 0.72645, 0.8608833333333331, 0.8951666666666667, 0.9088833333333334]
trial3=[0.5833833333333334, 0.7462333333333333, 0.8501999999999998, 0.8996999999999999, 0.9077333333333332]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)

[0.5716055555555556, 0.7413722222222111, 0.8536277777777777, 0.8988888888888887, 0.9076166666666666]


15 gives 0.9045833333333334
30 gives {None: 0.9011000000000001}
60 gives {None: 0.8973333333333333}

120 gives {None: 0.8980666666666668}
30-60 {None: 0.9031333333333332}

30-120 gives {None: 0.9072333333333333}

In [5]:
RP_vals = train_different_classes("RP_stagernet.pth", training_set, validation_set, 100, num_samples)

1
len of the dataloader is: 1
Start Training
[0.5416666666666666]
10
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888]
None
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888, 0.8611111111111112]


In [ ]:
{1: [0.9, 0.725, 0.675, 0.45, 0.5, 0.725, 0.65, 0.5, 0.4, 0.3, 0.55, 0.9, 0.475, 0.675, 0.525, 0.525, 0.85, 0.6, 0.7, 0.825, 0.575, 0.4, 0.375, 0.7, 0.55, 0.775, 0.65, 0.8, 0.75, 0.75, 0.825, 0.725], 10: [0.75, 0.875, 0.725, 0.725, 0.875, 0.825, 0.7, 0.875, 0.9, 0.725, 0.825, 0.775, 0.8, 0.85, 0.725, 0.675, 0.85, 0.725, 0.7, 0.725, 0.875, 0.825, 0.7, 0.85, 0.65, 0.5, 0.825, 0.925, 0.875, 0.575, 0.775, 0.975], 100: [0.775, 0.9, 0.875, 0.925, 0.975, 0.95, 0.875, 0.875, 0.95, 0.95, 0.775, 0.9, 0.875, 0.925, 0.825, 1.0, 0.875, 0.65, 0.75, 0.95, 0.975, 0.9, 0.875, 0.85, 0.95, 0.575, 0.9, 0.95, 0.825, 0.975, 0.9, 0.975], None: [0.9, 0.925, 0.975, 0.95, 0.95, 0.975, 0.95, 0.975, 1.0, 0.925, 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, 0.975, 0.875, 0.8, 0.9, 1.0, 0.875, 0.925, 0.875, 0.95, 0.8, 0.95, 0.95, 0.975, 1.0, 1.0, 0.95]}
Leaving out mouse number: 32.0

In [13]:
a=[1.0, 1.0, 1.0, 2, 3]
ind = np.unique(np.array(a))
print(ind)

[1. 2. 3.]


{1: 0.4689810606060606, 10: 0.6855946969696969, 100: 0.8547765151515151, 1000: 0.9181893939393939, None: 0.935465909090909}

[0.4689810606060606, 0.6855946969696969, 0.8547765151515151, 0.9181893939393939, 0.935465909090909]